In [3]:
import cv2
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

Setting up Selenium WebDriver

In [15]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get("https://k4.games/play/neon-rider/")
# https://k4.games/play/neon-rider/    (this is just a example of URL, this program doesn't have the logic of game)

Prepareing the input image by converting it to grayscale.

Applying edge detection to the preprocessed image to highlight the edges.

In [16]:
def capture_screen():
    screenshot = driver.get_screenshot_as_png()
    image = np.frombuffer(screenshot, dtype='uint8')
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (84, 84))
    normalized = resized / 255.0
    normalized = (normalized * 255).astype(np.uint8)
    return normalized
# Resize the image of game so that i will be easy to make decision easy.
def interpret_game_state(image):
    edges = cv2.Canny(image, threshold1=100, threshold2=200)
    return edges
    

LOGIC FOR THE GAME. 

IT MAY BE ANY GAME AND IT'S LOGIC

In [17]:
def make_decision(game_state):
    if np.sum(game_state[:, :42]) > np.sum(game_state[:, 42:]):
        return 'move_right'
    else:
        return 'move_left'

def perform_action(action):
    body = driver.find_element(By.TAG_NAME, 'body')
    if action == 'move_right':
        body.send_keys(Keys.ARROW_RIGHT)
    elif action == 'move_left':
        body.send_keys(Keys.ARROW_LEFT)

MAIN LOOP

In [18]:
num_iterations = 100  # Set the number of iterations
for _ in range(num_iterations):
    screen = capture_screen()
    preprocessed_image = preprocess_image(screen)
    game_state = interpret_game_state(preprocessed_image)
    action = make_decision(game_state)
    perform_action(action)
    time.sleep(0.1)  # Reduce the delay if necessary

# Close the browser after completing the iterations
driver.quit()